In [1]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span, DocBin

from IPython.display import Markdown, display
from IPython.display import clear_output

from itertools import combinations
import pandas as pd
from datetime import datetime

import os
import re
import json

C:\Users\justin\AppData\Local\Programs\Python\Python36\lib\site-packages\cupy\_environment.py:214: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  'CUDA path could not be detected.'
C:\Users\justin\AppData\Local\Programs\Python\Python36\lib\site-packages\cupy\_environment.py:214: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  'CUDA path could not be detected.'


In [12]:
spacy.prefer_gpu()
nlp = spacy.load("nl_core_news_lg")
df = pd.read_csv('..\\data\\ocred\\files_df.csv', index_col = 0)

## Dates

This notebook is used to awnser all rq's about dates. Those are 1.2 and 1.3

The first part is to awnser question 1.2

In [3]:
months = ['januari', 'februari', 'maart', 'april', 'mei', 'juni', 'juli', 'augustus', 'september', 'oktober', 'november', 'december',
         'january', 'february', 'march', 'april', 'may', 'june', 'juli', 'august', 'september', 'october', 'november', 'december',
         'jan', 'feb', 'mrt', 'apr', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
days = ['maandag', 'dinsdag', 'woensdag', 'donderdag', 'vrijdag', 'zaterdag', 'zondag',
       'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
sent = ['datum', 'verzonden', 'sent', 'date', 'received']

In [4]:
def printHilight(string):
    print('\x1b[1;31m'+string + ' ' +'\x1b[0m', end='')
    
def showMatches(doc, matches, regexMatches):   
    indexOfMatches = []
    for matchid, start, end in matches:
        for i in range(start, end):
            indexOfMatches.append(i)
            
    indexOfMatches = set(indexOfMatches)
    
    for token in doc:
        if token.i in indexOfMatches:
            printHilight(str(token.text))
        else:
            if token.text in regexMatches:
                printHilight(str(token.text))
            else:
                print(token, end=' ')
    
    return
    

In [5]:
def validate(dates, sep, pat):
    goodDates = []
    for date in dates:
        date = date.replace(' ', '')
        try:
            date = date.replace(sep, ' ')
            datetime.strptime(date, pat)
            goodDates.append(date.replace(' ', sep))
        except:
            try:
                if len(date.split(' ')) == 3 and len(date.split(' ')[2]) == 2:
                    datetime.strptime(date, '%d %m %y')
                    goodDates.append(date.replace(' ', sep))
            except:
                pass     
    return goodDates
            

def regexMatcher(text):
    results = []
    
    results += validate(re.findall('[0-3]{0,1}[0-9]\/[0-1]{0,1}[0-9]', text), '/', '%d %m')

    results += validate(re.findall('[0-3]{0,1}[0-9]\/[0-1]{0,1}[0-9]\/[0-9]{2,4}', text), '/', '%d %m %Y')

    results += validate(re.findall('[0-3]{0,1}[0-9]-[0-1]{0,1}[0-9]', text), '-', '%d %m')

    results += validate(re.findall('[0-3]{0,1}[0-9]-[0-1]{0,1}[0-9]-[0-9]{2,4}', text), '-', '%d %m %Y')
        
    return results

In [6]:
def inputHandling(message):
    while(True):
        i = input(message)
        if i == 'exit':
            return -1
        
        elif i == '':
            return 0
        
        try:
            i = int(i)
            return i
        
        except:
            print("input number, exit or nothing")
        

def evaluate(nlp):
    
    months = ['januari', 'februari', 'maart', 'april', 'mei', 'juni', 'juli', 'augustus', 'september', 'oktober', 'november', 'december',
         'january', 'february', 'march', 'april', 'may', 'june', 'juli', 'august', 'september', 'october', 'november', 'december',
         'jan', 'feb', 'mrt', 'apr', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    days = ['maandag', 'dinsdag', 'woensdag', 'donderdag', 'vrijdag', 'zaterdag', 'zondag',
           'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday',
           'ma', 'di', 'wo', 'woe', 'do', 'vrij', 'za', 'zat', 'zo', 'vr']
    
    datesPattern = [{"LOWER" : {"IN" : days}, "OP" : "?"}, 
           {"IS_DIGIT": True}, 
           {"LOWER" : {"IN" : months}},
           {"IS_PUNCT" : True, "OP" : "?", "TEXT":'.'},
           {"IS_DIGIT": True, "OP" : "?"}]
    matcher = Matcher(nlp.vocab)
    matcher.add("Dates", [datesPattern])
    
    # cor = exact match, inc = match is wrong (wrong bounds or label), mis = missing match, spu = found something that isnt a mactch
    if os.path.isfile('..\\data\\results\\dates_results,json'):
        with open('..\\data\\results\\dates_results,json') as f:
            results = json.load(f)
        
        pass
    else:
        results = {
            'correct':0,
            'incorrect':0,
            'missing':0,
            'spurious':0,
            'america' :0,
            'ocr':0,
            'oneoff':0,
            'other':0
        }
    
    print(results)
    cases = ['correct','incorrect','missing','spurious','america','ocr','oneoff','other']
    documents = 0
    
    while(True):
        try:
            print(sum([results[x] for x in results.keys() if x in cases[:4]]), results['documents'])
            sample = df.sample(1)
            print(sample.name.values[0], sample.page.values[0], '\n\n')
            text = sample.text.values[0]
            text = re.sub('\n+', '\n', text)
            text = re.sub(' +', ' ', text)
            doc = nlp(text)
            matches = matcher(doc)

            regexMatches = regexMatcher(text)

            showMatches(doc, matches, regexMatches)

            for case in cases:
                result = inputHandling(case)
                if result == -1:
                    results['documents'] += documents
                    return results
                else:
                    results[case] += result
            documents += 1
            clear_output()
        except:
            print('error')
            continue
            

In [9]:
def calcResults(r):
    precision = r['correct'] / (r['correct']+r['incorrect']+r['spurious'])
    recall = r['correct'] / (r['correct']+r['incorrect']+r['missing'])
    f1 = 2 * ((recall * precision)/(recall + precision))
    results['precision'] = precision
    results['recall'] = recall
    results['f1'] = f1 
    print(f'precision = {precision}, recall = {recall}, f1 = {f1}')
    return results

In [14]:
results = evaluate(nlp)
results = calcResults(results)

with open('..\\data\\results\\dates_results,json', 'w') as f:
    json.dump(results, f)


{'correct': 424, 'incorrect': 9, 'missing': 54, 'spurious': 13, 'america': 30, 'ocr': 19, 'oneoff': 12, 'other': 15, 'documents': 265, 'precision': 0.9506726457399103, 'recall': 0.8706365503080082, 'f1': 0.9088960342979635}
500 265
d8d9c5015c9ceb952052f29e1a27ed1f_openbaar-te-maken-documenten-deel-3 151 


Document 183 
 
 
 
 Van : 1020 
 Verzonden : maandag 15 juni 2020 15:43 
 Aan : HAGEM crisisteam corona D ! 
 e 
 e 
 e 
 e 
 m e 
 e 
 e 
 e 
 e 
 e 
 I 
 Onderwerp : Verslag crisisteam 15 juni 2020 
 Bijlagen : Verslag crisisteam 15 juni 2020.docx 
 Beste collega'’s , 
 Hierbij het verslag van het laatste Crisisteam . 
 Hartelijke groet , 
 mr , 
 Secretaris Crisisorganisatie Corona DJI 
 Beleidsadviseur Psychische Zorg 
 Dienst Justiriële Inrichtingen 
 Ministerie van Justitie en Veiligheid 
 
 Nederlands Instituut voor Forensische Psychiatrie en Psychologie ( NIFP ) 
 Herman Gorterstraat 5 , 3511 EW Utrecht | 7 ° etage 
 Postbus 13369 3507 LJ Utrecht 
 ! S 
 T A 
 EOS ! i ninius

In [15]:
results

{'correct': 424,
 'incorrect': 9,
 'missing': 54,
 'spurious': 13,
 'america': 30,
 'ocr': 19,
 'oneoff': 12,
 'other': 15,
 'documents': 265,
 'precision': 0.9506726457399103,
 'recall': 0.8706365503080082,
 'f1': 0.9088960342979635}

## conclusion

removed all access newlines and spaces

Part of the reason not all dates were caught was an oversight with american date formats. To validate the if a found match is actually a date, it needs a date format to compare the match to. Only the dd-mm-yyyy format was checked and not the american format of mm-dd-yyyy with the month before the year. This means it excludes dates like 04-14-2022 as this cannot be a date in the dd-mm-yyyy format because there obviously isn't a 14th month. In the american system this is just april 14th 2022.

Now there are also some dates that weren't found that should have been found that had nothing to do with american formats. Most of these are in a weird format or have extra spaces in between the number: "woensdag 27/5" and "3 -nov-2019" are examples of this. Also there are false positives that are actually false positives. Most of these are in the "31/12" format. These could all be actual dates but they could aslo be two random numbers seperated by a forward slash.

Another reason for not finding some dates is OCR mistakes. For example, in one case the date that was supposed to be found was "5/12/2021" but in the OCR process that string was read as "542/2021" where the "/" and "1" were seen as one character, a 4.

Missed dates in file names were excluded from the evaluation because these are not the dates your looking for

precision = 0.9197860962566845, recall = 0.8911917098445595, f1 = 0.9052631578947369

These next cells are for rq 1.3

In [ ]:
sent = ['datum', 'verzonden', 'sent', 'date']

Pattern = [{"LOWER" : {"IN" : sent}},
           {"IS_PUNCT" : True, "OP" : "?"},
           {"LOWER" : {"IN" : days}, "OP" : "?"}, 
           {"IS_DIGIT": True}, 
           {"LOWER" : {"IN" : months}},
           {"IS_DIGIT": True, "OP" : "?"}]

# also use regex for the american email date format thingy